In [79]:
import re

gpc_name = [
  'Residential: Fuel Combustion',
  'Residential: Grid Energy',
  'Commercial & Institutional: Fuel Combustion',
  'Commercial & Institutional: Grid Energy',
  'Manufacturing: Fuel Combustion',
  'Manufacturing: Grid Energy',
  'Energy: Fuel Combustion',
  'Energy: Grid Energy',
  'Ag, Forestry & Fishing: Fuel Combustion',
  'Ag, Forestry & Fishing: Grid Energy',
  'Non-Specified: Fuel Combustion',
  'Non-Specified: Grid Energy',
  'Fugitive Emissions: Coal',
  'Fugitive Emissions: Oil & Natural Gas',
  'On-Road Transport: Fuel Combustion',
  'On-Road Transport: Grid Energy',
  'Railways: Fuel Combustion',
  'Railways: Grid Energy',
  'Waterborne: Fuel Combustion',
  'Waterborne: Grid Energy',
  'Aviation: Fuel Combustion',
  'Aviation: Grid Energy',
  'Off-Road Transport: Fuel Combustion',
  'Off-Road Transport: Grid Energy',
  'Solid Waste: Disposed Within City',
  'Solid Waste: Disposed Outside City',
  'Biological Waste: Treated Within City',
  'Biological Waste: Treated Outside City',
  'Incineration: Treated Within City',
  'Incineration: Treated Outside City',
  'Wastewater: Treated Within City',
  'Wastewater: Treated Outside City',
]
gpc_sector = [
  'I.1.1',
  'I.1.2',
  'I.2.1',
  'I.2.2',
  'I.3.1',
  'I.3.2',
  'I.4.1',
  'I.4.2',
  'I.5.1',
  'I.5.2',
  'I.6.1',
  'I.6.2',
  'I.7.1',
  'I.8.1',
  'II.1.1',
  'II.1.2',
  'II.2.1',
  'II.2.2',
  'II.3.1',
  'II.3.2',
  'II.4.1',
  'II.4.2',
  'II.5.1',
  'II.5.2',
  'III.1.1',
  'III.1.2',
  'III.2.1',
  'III.2.2',
  'III.3.1',
  'III.3.2',
  'III.4.1',
  'III.4.2',
]
gpc_scope = [
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  1,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
]
gpc_3rd_level = [
  'residential_fuel',
  'residential_grid',
  'commercial_fuel',
  'commercial_grid',
  'manufacturing_fuel',
  'manufacturing_grid',
  'energy_fuel',
  'energy_grid',
  'agriculture_fuel',
  'agriculture_grid',
  'nonspecified_fuel',
  'nonspecified_grid',
  'fugitive_coal',
  'fugitive_oil',
  'onroad_fuel',
  'onroad_grid',
  'railways_fuel',
  'railways_grid',
  'waterborne_fuel',
  'waterborne_grid',
  'aviation_fuel',
  'aviation_grid',
  'offroad_fuel',
  'offroad_grid',
  'solidwaste_within',
  'solidwaste_outside',
  'biowaste_within',
  'biowaste_outside',
  'incineration_within',
  'incineration_outside',
  'wastewater_within',
  'wastewater_outside',
]
gpc_2nd_level = [
  'residential',
  'commercial',
  'manufacturing',
  'energy',
  'agriculture',
  'nonspecified',
  'fugitive',
  'onroad',
  'railways',
  'waterborne',
  'aviation',
  'offroad',
  'solidwaste',
  'biowaste',
  'incineration',
  'wastewater',
]
gpc_1st_level = ['stationary'] * 14 + ['transport'] * 10 + ['waste'] * 8
activities = [n + "_activity" for n in gpc_1st_level]
emission_factors = [n + "_emission_factor" for n in gpc_1st_level]


In [85]:
nodes = ["\n################## Emissions 3rd level"]
for i in range(len(gpc_scope)):
    nodes = nodes + [
        "- id: " + gpc_3rd_level[i] + "///  name: '" + gpc_sector[i] + " " + gpc_name[i] +
        "'///  type: simple.MultiplicativeNode///  quantity: emissions///  unit: kt/a///" +
        "  input_nodes: [" + activities[i] + ", " + emission_factors[i] +
        "]///  output_nodes: [" + [n + "_emissions" for n in gpc_2nd_level][int(i / 2)] + "]"
    ]
nodes = nodes + ["\n################## Emissions 2nd level"]
for i in range(len(gpc_2nd_level)):
    nodes = nodes + [
        "- id: " + gpc_2nd_level[i] + "_emissions///  name: '" +
        re.sub('\\..?$', '', gpc_sector[i * 2]) + " " +
        re.sub(':|Fuel Combustion|Grid Energy|Coal|Disposed |Within |City|Treated |Outside ', '', gpc_name[i * 2]) + "Emissions" +
        "'///  type: simple.AdditiveNode///  quantity: emissions///  unit: kt/a"
    ]
nodes = nodes + ["\n################### Emissions 1st level"]
nodes = nodes + [
    "- id: stationary_emissions///  name: I Stationary" +
    "///  type: simple.AdditiveNode///  quantity: emissions///  unit: kt/a///  input_nodes: " +
    "[residential_emissions, commercial_emissions, manufacturing_emissions, energy_emissions, agriculture_emissions, nonspecified_emissions, fugitive_emissions]" +
    "///  output_nodes: [net_emissions]"
]
nodes = nodes + [
    "- id: transport_emissions///  name: II Transport" +
    "///  type: simple.AdditiveNode///  quantity: emissions///  unit: kt/a///  input_nodes: " +
    "[onroad_emissions, railways_emissions, waterborne_emissions, aviation_emissions, offroad_emissions]" +
    "///  output_nodes: [net_emissions]"
]
nodes = nodes + [
    "- id: waste_emissions///  name: III Waste" +
    "///  type: simple.AdditiveNode///  quantity: emissions///  unit: kt/a///  input_nodes: " +
    "[solidwaste_emissions, biowaste_emissions, incineration_emissions, wastewater_emissions]" +
    "///  output_nodes: [net_emissions]"
]

# ="- id: "&$F2&"///  name: "&$A2&" "&REPLACE($B2, FIND(":", $B2),30,"")&"///  type: simple.AdditiveNode///  quantity: emissions///  unit: kt/a///  input_nodes: ["&$E2&", "&$E3&"]"

f = open("nodes.txt", "w")
for node in nodes:
    lines = node.split('///')
    for line in lines:
        print(line)
        f.write(line + "\n")
f.close()



################## Emissions 3rd level
- id: residential_fuel
  name: 'I.1.1 Residential: Fuel Combustion'
  type: simple.MultiplicativeNode
  quantity: emissions
  unit: kt/a
  input_nodes: [stationary_activity, stationary_emission_factor]
  output_nodes: [residential_emissions]
- id: residential_grid
  name: 'I.1.2 Residential: Grid Energy'
  type: simple.MultiplicativeNode
  quantity: emissions
  unit: kt/a
  input_nodes: [stationary_activity, stationary_emission_factor]
  output_nodes: [residential_emissions]
- id: commercial_fuel
  name: 'I.2.1 Commercial & Institutional: Fuel Combustion'
  type: simple.MultiplicativeNode
  quantity: emissions
  unit: kt/a
  input_nodes: [stationary_activity, stationary_emission_factor]
  output_nodes: [commercial_emissions]
- id: commercial_grid
  name: 'I.2.2 Commercial & Institutional: Grid Energy'
  type: simple.MultiplicativeNode
  quantity: emissions
  unit: kt/a
  input_nodes: [stationary_activity, stationary_emission_factor]
  output_node